In [1]:
import windfreak

In [138]:
mw_source.close()

In [54]:
low_f = 2820e6
high_f = 2920e6
low_f = 50e6
high_f = 100e6
n_steps = 3
scan_frequency_ranges = [low_f, high_f, n_steps] #change to start freq, stop freq and N step
scan_power = -10.0 #dBmW
freq_sample_time = 0.200 
#ata_rate = 10#Hz specifies the amount of time spent at each frequency in the sweep


In [55]:
scan_frequency_ranges

[50000000.0, 100000000.0, 3]

In [81]:
for x in ['model_type','serial_number','fw_version','hw_version','sub_version']:
    print(f'{x}: {mw_source.read(x)}')


model_type: WFT SynthHD 1694
serial_number: 1694
fw_version: Firmware Version 3.23
hw_version: Hardware Version 2.06
sub_version: HD


In [2]:
mw_source = windfreak.SynthHD('COM5')

In [3]:
mw_source._write('C1E0r0h0')
mw_source._write('C0E0r0h0')

In [381]:
mw_source._write('C0Y1') ## SETS the trigger polarity to HIGH Y0 --> LOW

In [139]:
print(mw_source[0].read('pll_power_on'))
print(mw_source[1].read('pll_power_on'))

AttributeError: 'NoneType' object has no attribute 'write'

In [47]:
print(mw_source[0].power)
print(mw_source[1].power)

-10.0
0.0


In [66]:
mw_source.trigger_mode = 'single frequency step'

In [60]:
mw_source._query('C0Y?')

'1'

In [136]:
mw_source[0].read('calibrated')

True

In [82]:
#we need to set up the RF pulser to enable its RF output on an external trigger
#and external trigger needs to set a voltage low (? -5V?, -3.3V? ... docs are confusing and suggest +3.3V)
#the trigger pulse width needs to be less than the time duration of each frequency during the sweep, 
#otherwise the RF will interpret the trigger pulse to move to the next frequency

frequency_step = (high_f - low_f) / (n_steps - 1)
mw_source.write('channel',0)
mw_source._write('Y1') ## SETS the trigger polarity Y1 to HIGH, Y0 to LOW. This function is undocumented!
mw_source[0].write('sweep_type',0) #X0 #linear sweep
mw_source[0].write('sweep_cont',0) #c0 #1 - set to do continuous sweep, 0 -- set to stop after full sweep
#mw_source[0].trigger_mode ='disabled' #w0
mw_source.trigger_mode = 'single frequency step' #w2

mw_source[0].write('sweep_direction',int(high_f >= low_f)) #^1 or ^0 if low_f > high_f

mw_source[0].write('sweep_freq_low', low_f / 1e6) #in MHz. Might need to be careful here on how many digits are written. for example, the device might not be happy with 10.000000000000000001
mw_source[0].write('sweep_freq_high', high_f / 1e6) #in MHz. Might need to be careful here on how many digits are written. for example, the device might not be happy with 10.000000000000000001
mw_source[0].write('sweep_freq_step', frequency_step / 1e6) #in MHz. Might need to be careful here on how many digits are written. for example, the device might not be happy with 10.000000000000000001

# mw_direct_device._write(f'l{low_f / 1e6:5.7f}')
# mw_direct_device._write(f'u{high_f / 1e6:5.7f}')
# mw_direct_device._write(f's{frequency_step / 1e6:5.7f}')

# set power
mw_source[0].write('power', scan_power) #f'W{scan_power:2.3f}'
mw_source[0].write('sweep_power_low',scan_power) #f'[{scan_power:2.3f}'
mw_source[0].write('sweep_power_high',scan_power) #f']{scan_power:2.3f}'

#freq_step_time = 1 / data_rate #in seconds
mw_source[0].write('sweep_time_step',freq_sample_time * 1e3) #limits between 4ms and 10000ms!

total_time_in_s = freq_sample_time * (n_steps)

print(f'Configured scan with')
print( f'scan_power = {scan_power}')
print(f'scan_frequencies (low to high) = {low_f*1e-6} - {high_f*1e-6} MHz')
print(f'frequency step size = {frequency_step*1e-6} MHz')
print(f'time @ each frequency = {freq_sample_time} s')
print(f'number of steps = {n_steps}')
print(f'time for full scan = {total_time_in_s} s')
print(f'trigger mode = {mw_source.trigger_mode}')


Configured scan with
scan_power = -10.0
scan_frequencies (low to high) = 50.0 - 100.0 MHz
frequency step size = 25.0 MHz
time @ each frequency = 0.2 s
number of steps = 3
time for full scan = 0.6000000000000001 s
trigger mode = single frequency step


In [84]:
mw_source[0].frequency = 65e6

In [116]:
mw_source._write('C0?')

In [137]:
mw_source._dev.readlines()

[]

In [109]:
mw_source._write('C1?')
status = mw_source._dev.readlines()

In [111]:
status = [x.decode('utf-8').strip() for x in status]

In [112]:
status

['C) Control Channel (A(0) or B(1))  1',
 'f) RF Frequency Now (MHz) 100.00000000, 6200.00000000',
 'W) RF Power (dBm) -10.000, 0.000',
 'V) Amp Calibration success? 1',
 'Z) Temperature Comp (0=none, 1=on set, 2=1sec, 3=10sec) 3, 3',
 'a) VGA DAC Setting (0=min, 4000=max) 636, 1016',
 '~) RF Phase Step (0=minimum, 360.0=maximum) 0.0000, 0.0000',
 'h) RF High(1) or Low(0) Power 1, 0',
 'E) PLL Chip En On(1) or Off(0) 1, 0',
 'U) PLL charge pump current 0, 5',
 'b) REF Doubler On(1) or Off(0) 0, 0',
 'i) Channel spacing (Hz) 100.000, 100.000',
 'x) Reference (external=0, int 27MHz=1, int 10MHz=2) 1',
 '*) PLL reference frequency (MHz) 27.00000000',
 'l) Sweep lower frequency (MHz) 50.00000000, 1000.00000000',
 'u) Sweep upper frequency (MHz) 100.00000000, 5000.00000000',
 's) Sweep step size (MHz/%) 25.00000000, 200.00000000',
 't) Sweep step time (mS) 200.000, 1.000',
 '[) Sweep amplitude low (dBm) -10.000, 0.000',
 ']) Sweep amplitude high (dBm) -10.000, 0.000',
 '^) Sweep direction (

In [115]:
mw_source[0].frequency = 65e6

In [92]:
mw_source[0].read('sweep_single')

False

In [119]:
mw_source.write('sweep_single',0)

In [71]:
print(mw_source[0].read('pll_power_on'))
print(mw_source[0].read('pa_power_on'))
print(mw_source[0].read('rf_enable'))

False
False
False


In [83]:
mw_source[0].enable = True

In [3]:
#we use the qcsapphire to deliver the trigger pulse
import qcsapphire

In [4]:
my_pulser = qcsapphire.Pulser('COM6')

SerialException: could not open port 'COM6': FileNotFoundError(2, 'The system cannot find the file specified.', None, 2)

In [16]:
my_pulser.pulse0.state(1)

'ok'

In [5]:
my_pulser.pulse1.state(0)
my_pulser.pulse2.state(0)
my_pulser.pulse3.state(0)
my_pulser.pulse4.state(0)

'ok'

In [14]:
my_pulser.pulse1.polarity('NORM')
my_pulser.pulse1.polarity()

'NORM'

'0.050000000'

In [135]:
my_pulser.pulse1.output.amplitude()

'5.00'

In [130]:
my_pulser.pulse1.output.amplitude(5.)

'ok'

In [18]:
my_pulser.pulse1.cmode('NORM')
my_pulser.pulse1.cmode()
# my_pulser.pulse1.cmode('BURST')
# print(my_pulser.pulse1.cmode())
# print(my_pulser.pulse1.bcounter(100))
my_pulser.pulse1.cmode('SINGLE')
my_pulser.pulse1.cmode()
my_pulser.pulse0.mode('SINGLE')

'ok'

In [7]:
my_pulser.pulse0.external.mode()

'DIS'

In [12]:
my_pulser.pulse1.state(1)

'ok'

In [19]:
#my_pulser.pulse0.period(.1)
print(f'system period: {my_pulser.pulse0.period()}')
#my_pulser.pulse1.width(0.05)
print(f'pulse 1 width: {my_pulser.pulse1.width()}')

system period: 0.100000000
pulse 1 width: 0.050000000


In [21]:
my_pulser.pulse0.external.mode('EXT')

Exception: Invalid parameter. Last write command == :PULSE0:EXTERNAL:MODE EXT

In [119]:
my_pulser.query('*RST')

'ok'

In [84]:
import time
for i in range(100):
    time.sleep(0.2)
    my_pulser.sof